In [3]:
def process(text:str):
    from parse import compile
    from string import punctuation
    p = compile("{hit:d}.")
    in_list = text.split()
    out_list = list()
    for seg in in_list:
        parse_result = p.parse(seg)
        if parse_result:
            # We got a number with a dot afterward:
            out_list.append(seg.lower())
        else:
            out_list.append(seg.translate(str.maketrans('', '', punctuation)).lower())
    return " ".join(out_list)


import os
def correct_path(s):
    basename = os.path.basename(s)
    current_dir = "/home/peterr/macocu/task8/transfer_10/"
    return os.path.join(current_dir, basename)
import pandas as pd
df = pd.read_json("ParlaSpeech-HR.v1.0.jsonl", orient="records", lines=True)
dev = df[df.split=="dev"]
test = df[df.split=="test"]

for df in [dev, test]:
    df["path"] = df.path.apply(correct_path)
    df["norm_text"] = df.norm_words.apply(" ".join)

In [4]:
test.head(2)

,path,orig_file,start,end,words,word_start_times,norm_words,norm_words_start_times,utterance_id_start,utterance_id_end,speaker_info,split,norm_words_edited,norm_text
841,/home/peterr/macocu/task8/transfer_10/dpKsbH3S...,"12 07 2017 - 1. dio, 4. sjednica, 9. saziv [dp...",4637.60,4644.80,"[koncem, godine, je, imalo, 44, zaposlenih, u,...","[0, 0.43, 0.91, 0.97, 1.35, 2.99, 4.78, 4.99, ...","[koncem, godine, je, imalo, četrdeset, četiri,...","[0, 0.43, 0.91, 0.97, 1.35, 2.51, 2.99, 4.78, ...",ParlaMint-HR_S04.u5878,ParlaMint-HR_S04.u5878,"{'Speaker_role': 'Regular', 'Speaker_type': 'n...",test,NaN,koncem godine je imalo četrdeset četiri zaposl...
1745,/home/peterr/macocu/task8/transfer_10/Y2U1PJH4...,"4 3 2020 - 16. sjednica, 9. saziv [Y2U1PJH4kNs...",20386.54,20406.15,"[Primjer, EU, odnosno, istaknuto, je, da, 18, ...","[0, 1.12, 2.26, 3.15, 3.66, 3.7199999999999998...","[primjer, eu, odnosno, istaknuto, je, da, osam...","[0, 1.12, 2.26, 3.15, 3.66, 3.7199999999999998...",ParlaMint-HR_S16.u5604,ParlaMint-HR_S16.u5604,"{'Speaker_role': 'Regular', 'Speaker_type': 'n...",test,NaN,primjer eu odnosno istaknuto je da osamnaest d...


In [6]:
from datasets import load_metric
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

import pandas as pd
import os
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer
import soundfile as sf
import os
# Use old or new vocab? 
os.system("cp vocab_300.json vocab.json")



tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
        "/home/peterr/macocu/task8/", unk_token="[UNK]", 
        pad_token="[PAD]", word_delimiter_token=" ")
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)
model = Wav2Vec2ForCTC.from_pretrained("/home/peterr/macocu/task8/25_normalised_200h/checkpoint-2720").cuda()

def transcribe(path):
    speech, sample_rate = sf.read(path)
    input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()

    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]).lower()
    return transcription

for df in [test, dev]:
    df["predicted"] = df.path.apply(transcribe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
for name, df in zip(["dev", "test"], [dev, test]):
    cer = cer_metric.compute(
        references = df.norm_text,
        predictions = df.predicted
    )
    wer = wer_metric.compute(
        references = df.norm_text,
        predictions = df.predicted
    )

    print(f"split: {name}, {cer=:0.4f}, {wer=:0.4f}")

split: dev, cer=0.0315, wer=0.1010
split: test, cer=0.0234, wer=0.0845


In [8]:
dev.to_csv("30_eval_results_dev.csv", index=False)
test.to_csv("30_eval_results_test.csv", index=False)

In [9]:
transcribe("sample_mono.wav")

'proglašava druga i oseba broa fita za predsjednika republike bez ograničenja trajanja mandata'